In [1]:
import pandas as pd

In [13]:
train_path = "mtl_data/train"
test_path = "mtl_data/test"
valid_path = "mtl_data/valid"

In [14]:
train_df = pd.read_csv(train_path+'/_classes.csv')
test_df = pd.read_csv(test_path+'/_classes.csv')
valid_df = pd.read_csv(valid_path+'/_classes.csv')

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1512 non-null   object
 1   0         1512 non-null   int64 
 2   1         1512 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 35.6+ KB


In [16]:
train_df.head()

,filename,0,1
0,9175_jpg.rf.368ca78302f0b4f070019c12f46dedbc.jpg,1,0
1,9169_jpg.rf.3dd91a23d55d01d56453a5f6763929e5.jpg,1,0
2,9176_jpg.rf.057b1239026f0cea2082612582c6e3db.jpg,1,0
3,10059_jpg.rf.0302b82e1c6443552752976970bcfaf7.jpg,1,0
4,9165_jpg.rf.11efabc84bb99e51dffc8553b21f9846.jpg,1,0


In [22]:
train_df['1']

0       0
1       0
2       0
3       0
4       0
       ..
1507    1
1508    1
1509    1
1510    1
1511    1
Name: 1, Length: 1512, dtype: int64

In [7]:
import os
import shutil

In [23]:
def organize_images(df, file_path):
    """
    Organizes image files into class-specific folders based on DataFrame annotations.

    Parameters:
    - df: pandas DataFrame with columns ['filename', 0, 1].
          'filename' contains image file names.
          Columns 0 and 1 are binary indicators for class membership.
    - file_path: String path to the directory containing the image files.
    """
    # Define the target directories
    class_dirs = {0: os.path.join(file_path, '0'), 1: os.path.join(file_path, '1')}

    # Create class directories if they don't exist
    for dir_path in class_dirs.values():
        os.makedirs(dir_path, exist_ok=True)

    # Iterate over DataFrame rows
    for _, row in df.iterrows():
        filename = row['filename']
        src_path = os.path.join(file_path, filename)

        # Check if the file exists before attempting to move
        if os.path.exists(src_path):
            # Determine the target class based on the DataFrame indicators
            if row[1] == 1:
                dst_path = class_dirs[1]
            else:
                dst_path = class_dirs[0]

            # Move the file to the appropriate class directory
            shutil.move(src_path, os.path.join(dst_path, filename))
        else:
            print(f"File not found: {src_path}")

In [24]:
organize_images(train_df,train_path)
organize_images(test_df,test_path)
organize_images(valid_df,valid_path)

In [33]:
import cv2
import mediapipe as mp

In [36]:
def crop_images(folder_path, output_folder="cropped_heads"):
    os.makedirs(output_folder, exist_ok=True)

    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        height, width, _ = image.shape

        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_image)

        if results.multi_face_landmarks:
            for i, face_landmarks in enumerate(results.multi_face_landmarks):
                # Get key landmarks for bounding box
                xs = [int(point.x * width) for point in face_landmarks.landmark]
                ys = [int(point.y * height) for point in face_landmarks.landmark]

                x_min = max(min(xs), 0)
                x_max = min(max(xs), width)
                y_min = max(min(ys), 0)
                y_max = min(max(ys), height)

                # Add margin to include forehead & below chin
                y_margin_top = int((y_max - y_min) * 0.3)
                y_margin_bottom = int((y_max - y_min) * 0.2)

                y_start = max(y_min - y_margin_top, 0)
                y_end = min(y_max + y_margin_bottom, height)

                cropped_face = image[y_start:y_end, x_min:x_max]

                # Skip tiny crops (bad detections)
                if cropped_face.shape[0] < 100 or cropped_face.shape[1] < 100:
                    continue

                output_filename = f"{os.path.splitext(filename)[0]}_head{i}.jpg"
                output_path = os.path.join(output_folder, output_filename)
                cv2.imwrite(output_path, cropped_face)

    print(f"Cropped heads saved to: {output_folder}")

In [37]:
crop_images('data/yawn_data/awake')

Cropped heads saved to: cropped_heads


In [38]:
crop_images('data/yawn_data/drowsy')

Cropped heads saved to: cropped_heads


In [2]:
import random
from PIL import Image

In [15]:
def augment_images(folder_path, target_count):
    # Get list of existing image files
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    current_count = len(image_files)
    if current_count >= target_count:
        print(f"Already have {current_count} images. No augmentation needed.")
        return

    print(f"Starting augmentation: {current_count} → {target_count} images")
    new_images_needed = target_count - current_count
    generated = 0

    while generated < new_images_needed:
        # Pick a random image
        img_name = random.choice(image_files)
        img_path = os.path.join(folder_path, img_name)

        # Open and rotate the image
        with Image.open(img_path) as img:
            angle = random.uniform(-25, 25)  # Rotation within ±60 degrees
            rotated = img.rotate(angle, resample=Image.BICUBIC, expand=False)

            # Randomly apply horizontal flip
            if random.random() < 0.5:
                rotated = rotated.transpose(Image.FLIP_LEFT_RIGHT)

            # Save with new name
            base_name, ext = os.path.splitext(img_name)
            new_name = f"{base_name}_aug_{generated}{ext}"
            rotated.save(os.path.join(folder_path, new_name))

        generated += 1

    print(f"✅ Augmentation complete: {target_count} images in total now.")

In [11]:
augment_images('data/drowsy_data/Drowsy',23000)

Starting augmentation: 22348 → 23000 images
✅ Augmentation complete: 23000 images in total now.


In [12]:
augment_images('data/drowsy_data/Non Drowsy',23000)

Starting augmentation: 19445 → 23000 images
✅ Augmentation complete: 23000 images in total now.


In [16]:
augment_images('data/yawn_data/awake',23000)

Starting augmentation: 1059 → 23000 images
✅ Augmentation complete: 23000 images in total now.


In [17]:
augment_images('data/yawn_data/drowsy',23000)

Starting augmentation: 1058 → 23000 images
✅ Augmentation complete: 23000 images in total now.
